In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing_main/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

WebIO._IJuliaInit()

In [3]:
h = 2
v_init_y = sqrt(2 * 9.81 * h)

6.26418390534633

In [4]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.009
N = 61
times = range(0, dt * (N - 1), length=N)
k_trans = 21
n = 15
m = 5

# Initial condition. Currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1] = -lb / 2.5                # xb
xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
xinit[3] = -30 * pi / 180           # theta
xinit[6] = -lb                      # x2
xinit[7] = 0.2                      # y2

xinit[9] = -v_init_y                # yb_dot
xinit[10] = -pi/2                   # theta_dot
xinit[14] = -1.0                    # y2_dot

init_mode = 1

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

In [5]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, k_trans, xterm, init_mode, dt);

In [6]:
# Objective
Q = Diagonal([10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 0.0])
R = Diagonal([1e-3, 1e-2, 1e-3, 1e-2, 0.0])

Qf = Q
# Qf = Diagonal([10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 0.0])

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, init_mode, k_trans, N, xinit, xterm);

In [8]:
# initialize guess
Xguess = [zeros(n) for x in Xref]
k_trans = nlp.k_trans

for k = 1:N
    if k <= k_trans
        Xguess[k] = xinit + (xterm - xinit) / (k_trans - 1) * (k - 1)
    else
        Xguess[k][1:14] = xterm[1:14]
    end
end

for k = 1:N-1
    if k < k_trans
        Xguess[k+1][end] = Xguess[k][end] + 0.001
    else
        Xguess[k+1][end] = Xguess[k][end] + 0.02
    end
end

In [13]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1253880
Number of nonzeros in inequality constraint Jacobian.:    74115
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1215
                     variables with only lower bounds:      120
                variables with lower and upper bounds:      121
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1032
Total number of inequality constraints...............:       61
        inequality constraints with only lower bounds:       61
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       

([-0.20000000000003493, 0.45355339059323285, -0.5235987755982847, -1.7424461934630155e-9, 2.780551258041212e-15, -0.4999999999999648, 0.2000000000000027, -4.790380523959208e-16, -6.264183905346336, -1.570796326794896  …  -0.5000000000000124, 9.01381372992354e-18, -2.1104077622749793e-15, 3.366424710350315e-15, -2.446223269304403e-15, 0.0, 0.0, 0.0, 0.0, 0.848539898959304], Ipopt.Optimizer)

In [14]:
Z_sol[1:15] - xinit

15-element Vector{Float64}:
 -3.4916514124461173e-14
 -4.0967229608668276e-14
  1.4099832412739488e-14
 -1.7424461934630155e-9
  2.780551258041212e-15
  3.519406988061746e-14
  2.6922908347160046e-15
 -4.790380523959208e-16
 -5.329070518200751e-15
  6.661338147750939e-16
  0.0
  0.0
  1.0416919756127546e-15
  8.881784197001252e-16
 -5.2722935594050705e-18

In [15]:
Z_sol[end-14:end] - xterm

15-element Vector{Float64}:
 -7.227551890309769e-14
 -2.55351295663786e-15
  3.5365892963064825e-15
  0.0
  8.873123143817921e-18
 -1.2434497875801753e-14
  9.01381372992354e-18
 -2.1104077622749793e-15
  3.366424710350315e-15
 -2.446223269304403e-15
  0.0
  0.0
  0.0
  0.0
  0.848539898959304

In [16]:
@show F1y_final = Z_sol[end-18]
@show F2y_final = Z_sol[end-16]
@show F1y_final + F2y_final

F1y_final = Z_sol[end - 18] = 44.56221189408092
F2y_final = Z_sol[end - 16] = 53.53778810591909
F1y_final + F2y_final = 98.10000000000001


98.10000000000001

In [ ]:
theta = zeros(N)
for k = 1:N
    theta[k] = Z_sol[3+20*(k-1)]
end
@show theta;

In [ ]:
dt = zeros(N-1)
for k = 1:N-1
    @show k
    @show dt[k] = Z_sol[20 + 20 * (k-1)]    
end

In [ ]:
# display(Z_sol)
writedlm("data_6.csv", Z_sol, ',')